In [1]:
using LimberJack
using GaussianProcess
using CSV
using DataFrames
using NPZ
using FITSIO
using PythonCall
using Interpolations
np = pyimport("numpy")

[ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
ERROR: LoadError: UndefVarError: ByRow not defined
Stacktrace:
 [1] include
   @ ./Base.jl:418 [inlined]
 [2] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::Nothing)
   @ Base ./loading.jl:1318
 [3] top-level scope
   @ none:1
 [4] eval
   @ ./boot.jl:373 [inlined]
 [5] eval(x::Expr)
   @ Base.MainInclude ./client.jl:453
 [6] top-level scope
   @ none:1
in expression starting at /home/jaimerz/.julia/packages/DataFrames/dgZn3/src/DataFrames.jl:1


LoadError: Failed to precompile DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0] to /home/jaimerz/.julia/compiled/v1.7/DataFrames/jl_v4mApm.

In [2]:
function get_gp_hp(chain;
                fid_cosmo=LimberJack.Cosmology(),
                latent_x=Vector(0:0.3:3))
    ###
    nsamples = length(chain[!, "h"])
    mu = fid_cosmo.Dz(vec(latent_x))
    eta = chain[!, "eta"]
    l = chain[!, "l"]
    σ8 = 0.81 * ones(nsamples)
    
    nx = 11
    vs = zeros(Float64, nx, nsamples)
    for i in 1:nx
        vs[i, :] = chain[!, string("v[", i ,"]")]
    end
    ###
    latent_x = Vector(0:0.3:3)
    x = Vector(range(0., stop=3., length=100))
    d = x[2] - x[1]
    gps = similar(vs)
    fs8s = zeros(Float64, 100, nsamples)
    for i in 1:length(l)
        K = sqexp_cov_fn(latent_x; eta=eta[i], l=l[i])
        gp = latent_GP(mu, vs[:, i], K)
        gps[:, i] = gp
        gp_cond = conditional(latent_x, x, gp, sqexp_cov_fn;
                              eta=1.0, l=l[i])
        Dzi = linear_interpolation(x, gp_cond ./ gp_cond[1], extrapolation_bc=Line())
        dDzs_mid = (gp_cond[2:end].-gp_cond[1:end-1])/d
        zs_mid = (x[2:end].+x[1:end-1])./2
        dDzi = linear_interpolation(zs_mid, dDzs_mid, extrapolation_bc=Line())
        dDzs_c = dDzi(x)
        fs8 = -σ8[i] .* (1 .+ x) .* dDzs_c
        fs8s[:, i] = fs8
    end
    
    return gps, fs8s
end

get_gp_hp (generic function with 1 method)

In [3]:
function get_lcdm_prods(chain)
    ###
    nsamples = length(chain[!, "h"])
    Ωm = chain[!, "Ωm"]
    Ωb = chain[!, "Ωb"]
    h = chain[!, "h"]
    s8 = chain[!, "s8"]
    ns = chain[!, "ns"]
    ###
    x = Vector(range(0., stop=3., length=100))
    Dzs = zeros(Float64, 100, nsamples)
    fs8s = zeros(Float64, 100, nsamples)
    for i in 1:nsamples
        cosmology = LimberJack.Cosmology(Ωm[i], Ωb[i], h[i], ns[i], s8[i],
                                         tk_mode="emulator",
                                         Pk_mode="Halofit")
        Dzs[:, i] = LimberJack.growth_factor(cosmology, x)
        fs8s[:, i] = LimberJack.fs8(cosmology, x)
    end
    
    return Dzs, fs8s
end

get_lcdm_prods (generic function with 1 method)

In [4]:
function get_gp(chain;
                   fid_cosmo=LimberJack.Cosmology(),
                   latent_x=Vector(0:0.3:3))
    ###
    nsamples = length(chain[!, "h"])
    mu = fid_cosmo.Dz(vec(latent_x))
    eta = 0.2 * ones(nsamples)
    l = 0.3 * ones(nsamples)
    σ8 = 0.81 * ones(nsamples)
    
    nx = 11
    vs = zeros(Float64, nx, nsamples)
    for i in 1:nx
        vs[i, :] = chain[!, string("v[", i ,"]")]
    end
    ###
    latent_x = Vector(0:0.3:3)
    x = Vector(range(0., stop=3., length=100))
    d = x[2] - x[1]
    gps = similar(vs)
    fs8s = zeros(Float64, 100, nsamples)
    for i in 1:length(l)
        K = sqexp_cov_fn(latent_x; eta=eta[i], l=l[i])
        gp = latent_GP(mu, vs[:, i], K)
        gps[:, i] = gp
        gp_cond = conditional(latent_x, x, gp, sqexp_cov_fn;
                              eta=1.0, l=l[i])
        Dzi = linear_interpolation(x, gp_cond ./ gp_cond[1], extrapolation_bc=Line())
        dDzs_mid = (gp_cond[2:end].-gp_cond[1:end-1])/d
        zs_mid = (x[2:end].+x[1:end-1])./2
        dDzi = linear_interpolation(zs_mid, dDzs_mid, extrapolation_bc=Line())
        dDzs_c = dDzi(x)
        fs8 = -σ8[i] .* (1 .+ x) .* dDzs_c
        fs8s[:, i] = fs8
    end
    
    return gps, fs8s
end

get_gp (generic function with 1 method)

In [5]:
function get_cls_SD(chain)

    data_set = "SD"
    meta = np.load(string("../data/", data_set, "/", data_set, "_meta.npz"))
    files = npzread(string("../data/", data_set, "/", data_set, "_files.npz"))

    tracers_names = pyconvert(Vector{String}, meta["tracers"])
    pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
    idx = pyconvert(Vector{Int}, meta["idx"])
    data_vector = pyconvert(Vector{Float64}, meta["cls"])
    cov_tot = pyconvert(Matrix{Float64}, meta["cov"]);
    
    #KiDS priors
    Ωm = chain[!, "Ωm"]
    Ωb = chain[!, "Ωb"]
    h = chain[!, "h"]
    s8 = chain[!, "s8"]
    ns = chain[!, "ns"]
    
    A_IA = chain[!, "A_IA"] 
    alpha_IA = chain[!, "alpha_IA"]

    eBOSS__0_0_b = chain[!, "eBOSS__0_0_b"]
    eBOSS__1_0_b = chain[!, "eBOSS__1_0_b"]
    
    DECALS__0_0_b = chain[!, "DECALS__0_0_b"]
    DECALS__1_0_b = chain[!, "DECALS__1_0_b"]
    DECALS__2_0_b = chain[!, "DECALS__2_0_b"]
    DECALS__3_0_b = chain[!, "DECALS__3_0_b"]
    DECALS__0_0_dz = chain[!, "DECALS__0_0_dz"]
    DECALS__1_0_dz = chain[!, "DECALS__1_0_dz"]
    DECALS__2_0_dz = chain[!, "DECALS__2_0_dz"]
    DECALS__3_0_dz = chain[!, "DECALS__3_0_dz"]
    
    KiDS1000__0_e_dz = chain[!, "KiDS1000__0_e_dz"]
    KiDS1000__1_e_dz = chain[!, "KiDS1000__1_e_dz"]
    KiDS1000__2_e_dz = chain[!, "KiDS1000__2_e_dz"]
    KiDS1000__3_e_dz = chain[!, "KiDS1000__3_e_dz"]
    KiDS1000__4_e_dz = chain[!, "KiDS1000__4_e_dz"]
    KiDS1000__0_e_m = chain[!, "KiDS1000__0_e_m"]
    KiDS1000__1_e_m = chain[!, "KiDS1000__1_e_m"]
    KiDS1000__2_e_m = chain[!, "KiDS1000__2_e_m"]
    KiDS1000__3_e_m = chain[!, "KiDS1000__3_e_m"]
    KiDS1000__4_e_m = chain[!, "KiDS1000__4_e_m"]

    cls = zeros(Float64, 610, length(h))
    for i in 1:length(h)
        nuisances = Dict("A_IA" => A_IA[i],
                         "alpha_IA" => alpha_IA[i],

                         "eBOSS__0_0_b" => eBOSS__0_0_b[i],
                         "eBOSS__1_0_b" => eBOSS__1_0_b[i],

                         "DECALS__0_0_b" => DECALS__0_0_b[i],
                         "DECALS__1_0_b" => DECALS__1_0_b[i],
                         "DECALS__2_0_b" => DECALS__2_0_b[i],
                         "DECALS__3_0_b" => DECALS__3_0_b[i],
                         "DECALS__0_0_dz" => DECALS__0_0_dz[i],
                         "DECALS__1_0_dz" => DECALS__1_0_dz[i],
                         "DECALS__2_0_dz" => DECALS__2_0_dz[i],
                         "DECALS__3_0_dz" => DECALS__3_0_dz[i],

                         "KiDS1000__0_e_dz" => KiDS1000__0_e_dz[i],
                         "KiDS1000__1_e_dz" => KiDS1000__1_e_dz[i],
                         "KiDS1000__2_e_dz" => KiDS1000__2_e_dz[i],
                         "KiDS1000__3_e_dz" => KiDS1000__3_e_dz[i],
                         "KiDS1000__4_e_dz" => KiDS1000__4_e_dz[i],
                         "KiDS1000__0_e_m" => KiDS1000__0_e_m[i],
                         "KiDS1000__1_e_m" => KiDS1000__1_e_m[i],
                         "KiDS1000__2_e_m" => KiDS1000__2_e_m[i],
                         "KiDS1000__3_e_m" => KiDS1000__3_e_m[i],
                         "KiDS1000__4_e_m" => KiDS1000__4_e_m[i])


        cosmology = LimberJack.Cosmology(Ωm[i], Ωb[i], h[i], ns[i], s8[i],
                                         tk_mode="emulator",
                                         Pk_mode="Halofit")
        cls[:, i] = Theory(cosmology, tracers_names, pairs,
                        idx, files; Nuisances=nuisances)
    end
    
    return cls
end;

In [6]:
function get_cls_SD_cosmo(chain)

    data_set = "SD"
    meta = np.load(string("../data/", data_set, "/", data_set, "_meta.npz"))
    files = npzread(string("../data/", data_set, "/", data_set, "_files.npz"))

    tracers_names = pyconvert(Vector{String}, meta["tracers"])
    pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
    idx = pyconvert(Vector{Int}, meta["idx"])
    data_vector = pyconvert(Vector{Float64}, meta["cls"])
    cov_tot = pyconvert(Matrix{Float64}, meta["cov"]);
    
    #KiDS priors
    Ωm = chain[!, "Ωm"]
    Ωb = chain[!, "Ωb"]
    h = chain[!, "h"]
    s8 = chain[!, "s8"]
    ns = chain[!, "ns"]
    

    cls = zeros(Float64, 610, length(h))
    for i in 1:length(h)

        cosmology = LimberJack.Cosmology(Ωm[i], Ωb[i], h[i], ns[i], s8[i],
                                         tk_mode="emulator",
                                         Pk_mode="Halofit")
        cls[:, i] = Theory(cosmology, tracers_names, pairs,
                        idx, files)
    end
    
    return cls
end;

In [7]:
function post_gp(fol_path)
    i = 1 
    while isfile(string(fol_path, "chain_", i, ".csv"))
        file_path = string(fol_path, "chain_", i, ".csv")
        chain = CSV.read(file_path, DataFrame)
        gp, fs8 = get_gp(chain)
        println(i)
        npzwrite(string(fol_path, "gp_", i ,".npz"), gp)
        npzwrite(string(fol_path, "fs8_", i ,".npz"), fs8)
        i += 1
    end    
end

post_gp (generic function with 1 method)

In [8]:
function post_gp_hp(fol_path)
    i = 1 
    while isfile(string(fol_path, "chain_", i, ".csv"))
        file_path = string(fol_path, "chain_", i, ".csv")
        chain = CSV.read(file_path, DataFrame)
        gp, fs8 = get_gp_hp(chain)
        println(i)
        npzwrite(string(fol_path, "gp_", i ,".npz"), gp)
        npzwrite(string(fol_path, "fs8_", i ,".npz"), fs8)
        i += 1
    end    
end

post_gp_hp (generic function with 1 method)

In [9]:
function post_lcdm(fol_path)
    i = 1 
    while isfile(string(fol_path, "chain_", i, ".csv"))
        file_path = string(fol_path, "chain_", i, ".csv")
        chain = CSV.read(file_path, DataFrame)
        Dz, fs8 = get_lcdm_prods(chain)
        println(i)
        npzwrite(string(fol_path, "Dz_", i ,".npz"), Dz)
        npzwrite(string(fol_path, "fs8_", i ,".npz"), fs8)
        i += 1
    end    
end

post_lcdm (generic function with 1 method)

In [10]:
function post_cls_SD(fol_path)
    i = 1 
    while isfile(string(fol_path, "chain_", i, ".csv"))
        file_path = string(fol_path, "chain_", i, ".csv")
        chain = CSV.read(file_path, DataFrame)
        cls = get_cls_SD(chain)
        println(i)
        npzwrite(string(fol_path, "cls_", i ,".npz"), cls)
        i += 1
    end    
end

post_cls_SD (generic function with 1 method)

In [11]:
function post_cls_SD_cosmo(fol_path)
    i = 1 
    while isfile(string(fol_path, "chain_", i, ".csv"))
        file_path = string(fol_path, "chain_", i, ".csv")
        chain = CSV.read(file_path, DataFrame)
        cls = get_cls_SD_cosmo(chain)
        println(i)
        npzwrite(string(fol_path, "cls_", i ,".npz"), cls)
        i += 1
    end    
end

post_cls_SD_cosmo (generic function with 1 method)

In [23]:
post_cls_SD("SD_TAP_0.6/")

1
2


In [32]:
post_cls_SD_cosmo("SD_cosmo_TAP_0.6/")

1
2
3
4
5
6


In [14]:
post_lcdm("ND_TAP_0.6/")

1
2
3
4
5


In [15]:
post_lcdm("ND_RSD_TAP_0.6/")

1
2
3
4
5
6


In [12]:
post_gp("../chains/ND_super_gp_Gibbs_TAP_0.6/")

LoadError: UndefVarError: DataFrame not defined

In [25]:
post_gp_hp("ND_RSD_gp_hp_TAP_0.6/")

1
2
3
4


In [18]:
post2("ND_gp_hp_TAP_0.6/")

1


In [70]:
post2("DES_full_gp_nos8_2_TAP_0.6/")

1
2
